# 💼 Financial Performance Analysis
## Exploratory Data Analysis & Forecasting

**Author:** David Madison  
**Date:** November 2025

This notebook performs exploratory analysis on financial data and develops forecasting models.

## 1. Import Libraries & Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')

# Styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries loaded successfully")

In [ ]:
# Load financial data
df = pd.read_csv('../data/financials.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

print(f"Dataset Shape: {df.shape}")
print(f"Date Range: {df['Date'].min().strftime('%B %Y')} to {df['Date'].max().strftime('%B %Y')}")
print(f"\nFirst 5 rows:")
df.head()

## 2. Data Overview & Quality Check

In [ ]:
# Basic statistics
print("📊 Dataset Information:")
print(df.info())
print("\n📈 Descriptive Statistics:")
df.describe()

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())

# Check data types
print("\nData Types:")
print(df.dtypes)

## 3. Key Performance Indicators (KPIs)

In [ ]:
# Calculate KPIs
print("💰 FINANCIAL KPIs")
print("=" * 50)
print(f"Total Revenue:        ${df['Revenue'].sum():,.2f}")
print(f"Total Expenses:       ${df['Expenses'].sum():,.2f}")
print(f"Total Net Profit:     ${df['Net_Profit'].sum():,.2f}")
print(f"Average Profit Margin: {df['Profit_Margin'].mean():.2f}%")
print(f"\nAverage Monthly Revenue: ${df['Revenue'].mean():,.2f}")
print(f"Average Monthly Profit:  ${df['Net_Profit'].mean():,.2f}")
print(f"\nRevenue Growth Rate: {((df['Revenue'].iloc[-1] / df['Revenue'].iloc[0]) - 1) * 100:.2f}%")

## 4. Revenue & Expense Analysis

In [ ]:
# Plot revenue vs expenses
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df['Date'], df['Revenue'], label='Revenue', linewidth=2.5, color='#2E7D32')
ax.plot(df['Date'], df['Expenses'], label='Expenses', linewidth=2.5, color='#C62828')
ax.fill_between(df['Date'], df['Revenue'], df['Expenses'], 
                where=(df['Revenue'] >= df['Expenses']), 
                alpha=0.2, color='#2E7D32', label='Profit Zone')

ax.set_xlabel('Date', fontsize=12, fontweight='bold')
ax.set_ylabel('Amount ($)', fontsize=12, fontweight='bold')
ax.set_title('Revenue vs Expenses Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=11)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Profit margin trend
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df['Date'], df['Profit_Margin'], linewidth=2.5, color='#1565C0', marker='o', markersize=4)
ax.axhline(y=df['Profit_Margin'].mean(), color='#FF6F00', linestyle='--', 
           linewidth=2, label=f'Average: {df["Profit_Margin"].mean():.2f}%')

ax.set_xlabel('Date', fontsize=12, fontweight='bold')
ax.set_ylabel('Profit Margin (%)', fontsize=12, fontweight='bold')
ax.set_title('Profit Margin Trend', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Expense Breakdown Analysis

In [ ]:
# Pie chart of average expenses
avg_marketing = df['Marketing_Cost'].mean()
avg_operational = df['Operational_Cost'].mean()

fig, ax = plt.subplots(figsize=(8, 8))
colors = ['#FF6F00', '#1565C0']
explode = (0.05, 0.05)

ax.pie([avg_marketing, avg_operational], 
       labels=['Marketing', 'Operational'], 
       autopct='%1.1f%%', 
       startangle=90, 
       colors=colors,
       explode=explode,
       textprops={'fontsize': 12, 'fontweight': 'bold'})

ax.set_title('Average Monthly Expense Breakdown', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\nAverage Marketing Cost: ${avg_marketing:,.2f} ({avg_marketing/(avg_marketing+avg_operational)*100:.1f}%)")
print(f"Average Operational Cost: ${avg_operational:,.2f} ({avg_operational/(avg_marketing+avg_operational)*100:.1f}%)")

## 6. Seasonality Analysis

In [ ]:
# Monthly averages
monthly_avg = df.groupby('Month').agg({
    'Revenue': 'mean',
    'Net_Profit': 'mean',
    'Profit_Margin': 'mean'
}).reset_index()

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_avg['Month_Name'] = monthly_avg['Month'].apply(lambda x: month_names[x-1])

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Revenue by month
ax1.bar(monthly_avg['Month_Name'], monthly_avg['Revenue'], color='#2E7D32')
ax1.set_xlabel('Month', fontsize=11, fontweight='bold')
ax1.set_ylabel('Average Revenue ($)', fontsize=11, fontweight='bold')
ax1.set_title('Average Revenue by Month', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Profit margin by month
ax2.bar(monthly_avg['Month_Name'], monthly_avg['Profit_Margin'], color='#1565C0')
ax2.set_xlabel('Month', fontsize=11, fontweight='bold')
ax2.set_ylabel('Average Profit Margin (%)', fontsize=11, fontweight='bold')
ax2.set_title('Average Profit Margin by Month', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Anomaly Detection

In [ ]:
# Detect profit anomalies using 2-sigma rule
profit_mean = df['Net_Profit'].mean()
profit_std = df['Net_Profit'].std()

# Flag anomalies
df['Is_Anomaly'] = np.abs(df['Net_Profit'] - profit_mean) > 2 * profit_std
anomalies = df[df['Is_Anomaly']]

print(f"⚠️ Anomalies Detected: {len(anomalies)} months\n")
for _, row in anomalies.iterrows():
    print(f"{row['Date'].strftime('%B %Y')}: ${row['Net_Profit']:,.2f} profit (Margin: {row['Profit_Margin']:.2f}%)")

# Visualize
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df['Date'], df['Net_Profit'], linewidth=2, color='#1565C0', label='Net Profit')
ax.scatter(anomalies['Date'], anomalies['Net_Profit'], 
           color='red', s=100, zorder=5, label='Anomalies')
ax.axhline(y=profit_mean, color='green', linestyle='--', label='Mean')
ax.axhline(y=profit_mean + 2*profit_std, color='orange', linestyle=':', label='+2σ')
ax.axhline(y=profit_mean - 2*profit_std, color='orange', linestyle=':', label='-2σ')

ax.set_xlabel('Date', fontsize=12, fontweight='bold')
ax.set_ylabel('Net Profit ($)', fontsize=12, fontweight='bold')
ax.set_title('Profit Anomaly Detection (2-Sigma Rule)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Time-Series Forecasting with Prophet

In [ ]:
# Prepare data for Prophet
prophet_df = df[['Date', 'Revenue']].copy()
prophet_df.columns = ['ds', 'y']

# Train model
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.05
)
model.fit(prophet_df)

print("✅ Prophet model trained successfully")

In [ ]:
# Generate 12-month forecast
future = model.make_future_dataframe(periods=12, freq='MS')
forecast = model.predict(future)

# Plot forecast
fig = model.plot(forecast, figsize=(14, 6))
ax = fig.gca()
ax.set_xlabel('Date', fontsize=12, fontweight='bold')
ax.set_ylabel('Revenue ($)', fontsize=12, fontweight='bold')
ax.set_title('Revenue Forecast (12 Months)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Plot forecast components
fig = model.plot_components(forecast, figsize=(14, 8))
plt.tight_layout()
plt.show()

In [ ]:
# Display forecast summary
forecast_new = forecast.tail(12)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_new.columns = ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']
forecast_new['Date'] = forecast_new['Date'].dt.strftime('%B %Y')

print("📊 12-Month Revenue Forecast:\n")
print(forecast_new.to_string(index=False))

print(f"\n💰 Total Forecasted Revenue: ${forecast_new['Forecast'].sum():,.2f}")

## 9. Model Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# Use last 6 months as test set
train_df = prophet_df[:-6]
test_df = prophet_df[-6:]

# Retrain on training set
model_eval = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.05
)
model_eval.fit(train_df)

# Predict on test set
test_predictions = model_eval.predict(test_df)

# Calculate metrics
mae = mean_absolute_error(test_df['y'], test_predictions['yhat'])
rmse = np.sqrt(mean_squared_error(test_df['y'], test_predictions['yhat']))
mape = mean_absolute_percentage_error(test_df['y'], test_predictions['yhat']) * 100

print("🎯 Model Performance (Last 6 Months):")
print("=" * 40)
print(f"MAE (Mean Absolute Error):  ${mae:,.2f}")
print(f"RMSE (Root Mean Squared Error): ${rmse:,.2f}")
print(f"MAPE (Mean Absolute % Error):   {mape:.2f}%")
print("\n✅ MAPE < 5% indicates excellent forecast accuracy")

## 10. Summary & Insights

In [ ]:
print("📋 EXECUTIVE SUMMARY")
print("=" * 70)
print(f"\n📊 Historical Performance ({len(df)} months):")
print(f"   Total Revenue:    ${df['Revenue'].sum():,.2f}")
print(f"   Total Profit:     ${df['Net_Profit'].sum():,.2f}")
print(f"   Avg Margin:       {df['Profit_Margin'].mean():.2f}%")
print(f"   Revenue Growth:   {((df['Revenue'].iloc[-1] / df['Revenue'].iloc[0]) - 1) * 100:.2f}%")

forecast_revenue = forecast.tail(12)['yhat'].sum()
print(f"\n🔮 12-Month Forecast:")
print(f"   Expected Revenue: ${forecast_revenue:,.2f}")
print(f"   Model Accuracy:   MAPE {mape:.2f}%")

print(f"\n⚠️ Key Findings:")
print(f"   - {len(anomalies)} anomalous months detected requiring investigation")
print(f"   - Seasonal patterns show Q4 revenue boost")
print(f"   - Current expense ratio: {(df['Expenses'].iloc[-1] / df['Revenue'].iloc[-1]) * 100:.1f}%")
print(f"\n💡 Recommendations:")
print(f"   1. Investigate expense spikes in flagged months")
print(f"   2. Optimize marketing spend during low-performing months")
print(f"   3. Implement cost controls if expense ratio exceeds 90%")
print(f"   4. Leverage seasonal trends for strategic planning")

print("\n" + "=" * 70)
print("✅ Analysis Complete")